In [13]:
import pandas as pd
import json
import os
import numpy as np
import ast
import tqdm

In [4]:
data_path = '/ssd2/sanghoon/data/EHR/mimic3/mimic_extract/sh_pre_los100/24h_6g/'
mimic_path='/ssd2/sanghoon/data/EHR/mimic3/mimic3/'
patitentTM_preprocessing_path='/home/sanghoon/drug/project/EHR/PatientTM/data/extended/preprocessing/'
labels=pd.read_csv(data_path+'label_last_visit.csv')
max_len_text_r =pd.read_pickle(data_path+'for_bert_text.pkl')

In [5]:
drg_code = pd.read_csv(mimic_path+'DRGCODES.csv')
ccs_code = pd.read_json(patitentTM_preprocessing_path+'ICDandCCSmappings/merged_icdccs_codes.json', typ='series')
ccs_code=ccs_code.reset_index()
diag=pd.read_csv(mimic_path+'DIAGNOSES_ICD.csv')

In [6]:
diag=diag[diag['HADM_ID'].isin(labels['hadm_id'].unique())]
pre_data=pd.read_csv('/ssd2/sanghoon/data/EHR/MP_EHR/preprocessed.csv')
labels=labels[labels['have_note']==1]
pre_data=pre_data[pre_data['HADM_ID'].isin(labels['hadm_id'])]


In [7]:
all_diag_ccs = pre_data['DIAG_CCS'].apply(lambda x: ast.literal_eval(x))

In [8]:
all_d_label = list(set(np.concatenate(all_diag_ccs.values)))

In [10]:
d_ccs_label_dict=dict(zip(all_d_label, [i for i in range(len(all_d_label))]))
len(d_ccs_label_dict)

251

In [11]:
def  make_label_idx_list(t):
    
    return list(map(lambda x: d_ccs_label_dict[x],t))

In [14]:
label_dict={}
for i in tqdm.tqdm(range(len(pre_data))):
    key = pre_data.iloc[i]['HADM_ID']
    label_t = list(map(lambda x : d_ccs_label_dict[x] ,all_diag_ccs.iloc[i]))
    temp_label=np.zeros([1,len(d_ccs_label_dict)])
    temp_label[0][label_t]=1
    label_dict[key]=temp_label
    

100%|█████████████████████████████████████████████████████████████████| 30060/30060 [00:02<00:00, 13877.02it/s]


In [ ]:
pd.to_pickle(label_dict, data_path+'/ccs_label.pkl')